using:
- subject-wise normalization
- segment length of 200

In [2]:
import tensorflow as tf
from tensorflow.keras import models, Model
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.layers import Dense, Conv2D, Flatten, MaxPool2D, BatchNormalization, Dropout, Input, Concatenate, GlobalAvgPool2D
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.utils import Sequence
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint, Callback
from sklearn.metrics import accuracy_score, balanced_accuracy_score, f1_score, precision_score, recall_score, confusion_matrix, ConfusionMatrixDisplay
import numpy as np
import matplotlib.pyplot as plt
import os
import json

In [21]:
def evaluator(y_pred, y_true, verbose=False):
    """Returns evaluation metric scores"""
    accuracy = accuracy_score(y_pred=y_pred, y_true=y_true)
    balanced_accuracy = balanced_accuracy_score(y_pred=y_pred, y_true=y_true)
    f1 = f1_score(y_pred=y_pred, y_true=y_true, average='weighted')
    recall = recall_score(y_pred=y_pred, y_true=y_true, average='weighted')
    precision = precision_score(y_pred=y_pred, y_true=y_true, average='weighted')
    confusion = confusion_matrix(y_pred=y_pred, y_true=y_true)

    # display scores
    if verbose:
        ConfusionMatrixDisplay(confusion_matrix=confusion, display_labels=[False, True]).plot(cmap=plt.cm.Blues)
        plt.title('Physical fatigue')

        print(f'accuracy: {accuracy}\n'
              f'balanced accuracy: {balanced_accuracy}\n'
              f'f1 (weighted): {f1}\n'
              f'recall (weighted): {recall}\n'
              f'precision (weighted): {precision}')

    return {'accuracy': accuracy,
            'balanced_accuracy': balanced_accuracy,
            'f1': f1,
            'recall': recall,
            'precision': precision}

def test_set_evaluator(model, path, test_indices, verbose=True):
    # load + predict test data
    y_true = np.empty(len(test_indices), dtype=int)
    y_pred = np.empty(len(test_indices), dtype=float)

    for i, index in enumerate(test_indices):
        X_i = np.load(path + f'/feature_vector{index}.npy', allow_pickle=True)
        y_true_i = np.load(path + f'/labels{index}.npy', allow_pickle=True)[variable]

        X_i = tf.expand_dims(X_i, axis=0) # add "batch dimension"
        logits_pred_i = model.predict(X_i)

        y_true[i] = y_true_i
        y_pred[i] = logits_pred_i

    y_probs = tf.math.sigmoid(y_pred)
    y_pred = tf.round(y_probs)

    # print metrics
    scores = evaluator(y_pred, y_true, verbose=verbose)

    # plot distribution of predictions
    if verbose:
        plt.subplots_adjust(left=0.1,
                            bottom=0.01,
                            right=1.2,
                            top=0.6,
                            wspace=0.4,
                            hspace=0.4)

        # predicted probabilities
        plt.subplot(1, 2, 1)
        plt.hist(y_probs)
        plt.title('P(y == 1)')
        plt.vlines(x=0.5, ymin=0, ymax=len(y_probs), color='red')

        # predicted labels
        plt.subplot(1, 2, 2)
        plt.hist(y_pred)
        plt.title('Labels')

        plt.show()

    return scores

In [4]:
VARIABLES = ['ActivityCounts', 'Barometer', 'BloodPerfusion',
             'BloodPulseWave', 'EnergyExpenditure', 'HR',
             'HRV', 'RESP', 'Steps', 'SkinTemperature']

In [5]:
GRAYSCALE = False # grayscale or rgb

# Import data

In [6]:
variable = {'phF': 0, 'MF': 1}['phF'] # TODO: for now just classify one variable

In [7]:
path = './Output'

In [8]:
# dimensions
N, HEIGHT, WIDTH, CHANNELS = sum([1 for p in os.listdir(path) if (p[:14] == 'feature_vector' and p[:19] != 'feature_vector_stat')]), \
                             *np.load(path + '/feature_vector0.npy').shape
CHANNELS = len(VARIABLES) if GRAYSCALE else CHANNELS # reduce channels for grayscale

# metadata (subjectID etc.)
with open(path + '/metadata.txt') as f:
    metadata = f.read()
metadata = json.loads(metadata.replace('\'', '\"').replace('False', 'false').replace('True', 'true')) # doesn't accept other chars

In [9]:
print(f'Total datapoints: {N}')

Total datapoints: 1218


# Define data generator

In [10]:
# image-wise
def rgb2gray(rgb):
    """greyscale = 0.2989 * red + 0.5870 * green + 0.1140 * blue"""
    return np.dot(rgb[:, :, :3], [0.2989, 0.5870, 0.1140])

In [11]:
class DataGenerator(Sequence):

    def __init__(self, data_path: str, indices_dataset: list, batch_size=32, dim=(HEIGHT, WIDTH), n_channels=CHANNELS, shuffle=True):
        self.data_path = data_path # path to full dataset
        self.dim = dim # image dimension
        self.batch_size = batch_size
        self.indices_dataset = indices_dataset # indices of full dataset (different for train/validation/test set)
        self.n_channels = n_channels
        self.shuffle = shuffle

        self.on_epoch_end() # shuffle data for each epoch

    def on_epoch_end(self):
        """
        Shuffle data for each epoch
        """
        if self.shuffle:
            np.random.shuffle(self.indices_dataset)

    def __data_generation(self, indices):
        """
        Loads and returns datapoints[indices]
        """
        # init
        X = np.empty((self.batch_size, *self.dim, self.n_channels))
        y = np.empty(self.batch_size, dtype=float) # TODO: int for non-logits

        # load individual datapoints
        for i, index in enumerate(indices):
            images = np.load(self.data_path + f'/feature_vector{index}.npy', allow_pickle=True)
            if GRAYSCALE:
                images_gray = np.empty((HEIGHT, WIDTH, self.n_channels))
                for j in range(len(VARIABLES)):
                    image_rgb = images[:, :, (3 * j): (3 * (j + 1))]
                    image_gray = rgb2gray(image_rgb)
                    images_gray[:, :, j] = image_gray
                images = images_gray

            X[i, ] = images
            y[i] = np.load(self.data_path + f'/labels{index}.npy', allow_pickle=True)[variable] # TODO: for now just one variable

        return X, y

    def __len__(self):
        """
        Number of batches per epoch
        """
        return int(np.floor(len(self.indices_dataset) / self.batch_size))

    def __getitem__(self, index):
        """
        Generates batch[index]
        """
        # calculate indices of batch
        indices = self.indices_dataset[index * self.batch_size:(index + 1) * self.batch_size]

        # generate batch
        X, y = self.__data_generation(indices)

        return X, y

# CNN

Data generators

In [12]:
'''training_batch_generator = DataGenerator(data_path=path, indices_dataset=training_indices, batch_size=16)'''

'training_batch_generator = DataGenerator(data_path=path, indices_dataset=training_indices, batch_size=16)'

Loss function

In [13]:
# loss function
def weighted_cross_entropy(weight):
    def weighted_cross_entropy_with_logits(labels, logits):
        loss = tf.nn.weighted_cross_entropy_with_logits(
            labels, logits, weight
        )
        return loss
    return weighted_cross_entropy_with_logits

# weight (imbalanced classes)
def check_imbalance(path_to_labels, indices):
    """Returns indices of positives/negatives"""
    y = np.empty((len(indices), 2), dtype=int)
    for i, index in enumerate(indices):
        y[i, ] = np.load(path_to_labels + f'/labels{index}.npy', allow_pickle=True)

    positives = np.where(y[:, variable] == 1)[0] # TODO: for now just one variable
    negatives = np.where(y[:, variable] == 0)[0] # TODO: for now just one variable

    return np.array(indices)[positives], np.array(indices)[negatives]

def get_weighting_factor(path, train_set_indices):
    positives, negatives = check_imbalance(path, train_set_indices)
    sample_weight = len(negatives) / len(positives) # for weighted cross-entropy
    return sample_weight

Model

In [14]:
# TODO: make possible for grayscale
class CNN(tf.keras.Model):

    def __init__(self, name='custom_CNN', **kwargs):
        super(CNN, self).__init__(name, **kwargs)

        self.in_shape = (HEIGHT, WIDTH, CHANNELS)
        self.in_shape_mobilenet = (HEIGHT, WIDTH, 3)

        # MobileNetV2 embedding
        self.mobilenet = MobileNetV2(input_shape=self.in_shape_mobilenet, weights='imagenet', include_top=False)
        self.mobilenet._name = 'mobilenet'
        self.mobilenet.trainable = False
        self.finetuning = False
        self.out_shape_mobilenet = self.mobilenet.layers[-1].output_shape # for one spectrogram

        # Concatenation
        self.concat = Concatenate(name='concat')

        # Global pooling
        self.pool = GlobalAvgPool2D(name='global_avg_pool')

        # TODO: more sophisticated dense (dropout, regularizer, init., ...)
        # Fully-connected network
        self.flatten = Flatten(name='flatten', input_shape=(self.out_shape_mobilenet * (CHANNELS // 3), ))
        self.dense = Dense(1, name='dense') # keep logits
        self.out_shape = 1

        # build graph
        self.build_graph()

    def build_graph(self):
        self.build(input_shape=(None, *self.in_shape))
        x = Input(shape=self.in_shape)
        Model(inputs=[x], outputs=self.call(x))

    def set_finetuning(self, mode=True):
        self.finetuning = mode
        self.mobilenet.trainable = mode

        for layers in self.mobilenet.layers:
            layers.trainable = False

        # "activate" last conv layer of MobileNet
        self.mobilenet.layers[-3].trainable = mode
        self.mobilenet.layers[-2].trainable = mode

    def call(self, inputs):
        """
        Model predictions (logits)
        :param inputs: all spectrograms of shape (HEIGHT, WIDTH, CHANNELS)
        :return: class prediction (logits)
        """
        # MobileNetV2 embeddings
        x = [self.mobilenet(inputs[..., i:i+3], training=self.finetuning) for i in range(0, CHANNELS, 3)]

        # Concatenation
        x = self.concat(x)

        # Global pooling
        x = self.pool(x)

        # Fully-connected network
        x = self.flatten(x)
        x = self.dense(x)

        return x

# Stratified Group k-Fold CV

### TODO: augmentations in CV?

In [15]:
from sklearn.model_selection import StratifiedGroupKFold
from tqdm import tqdm
import warnings

In [18]:
subjects = [meta['subjectID'] for meta in metadata]
subjects

[1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 5,
 5,
 5,
 5,
 5,
 5,
 5,
 5,
 5,
 5,
 5,
 7,
 7,
 7,
 7,
 7,
 8,
 8,
 8,
 8,
 8,
 8,
 8,
 8,
 8,
 8,
 8,
 8,
 8,
 8,
 8,
 8,
 8,
 8,
 8,
 9,
 9,
 9,
 9,
 9,
 9,
 9,
 9,
 9,
 9,
 9,
 9,
 9,
 9,
 9,
 9,
 9,
 9,
 9,
 9,
 10,
 10,
 10,
 10,
 10,
 10,
 10,
 10,
 10,
 10,
 10,
 10,
 10,
 10,
 11,
 11,
 11,
 11,
 11,
 11,
 11,
 11,
 11,
 11,
 11,
 11,
 11,
 11,
 11,
 11,
 11,
 11,
 11,
 12,
 12,
 12,
 12,
 12,
 12,
 12,
 12,
 12,
 12,
 12,
 12,
 12,
 12,
 12,
 12,
 12,
 12,
 12,
 12,
 13,
 13,
 13,
 13,
 13,
 13,
 13,
 13,
 13,
 13,
 13,
 13,
 13,
 13,
 13,
 13,
 13,
 13,
 14,
 14,
 14,
 14,
 14,
 14,
 14,
 14,
 14,
 14,
 14,
 14,
 14,
 14,
 14,
 14,
 15,
 15,
 15,
 15,
 15,
 15,
 15,
 15,
 15,
 15,
 16,
 16,
 16,
 16,
 16,
 16,
 16,
 16,
 16,
 16,
 16,
 16,
 16,
 16,
 

In [25]:
# variables
folds = 5
indices = np.arange(N)
epochs = 1
learning_rate = 1e-3
batch_size = 16

# load labels
y = np.empty(N, dtype=int)
for i in range(N):
    y[i] = np.load(path + f'/labels{i}.npy', allow_pickle=True)[variable] # TODO: multiclass

# CV
cv = StratifiedGroupKFold(n_splits=folds)
scores_cv = []

print(f'Starting cross-validation for physical fatigue')
with tqdm(total=folds) as pbar:
    for i, (train_indices, test_indices) in enumerate(cv.split(indices, y, groups=subjects)):
        # training set
        train_dataloader = DataGenerator(path, train_indices, batch_size=batch_size)

        # weights for loss function
        sample_weights = get_weighting_factor(path, train_indices)

        # model
        model = CNN()
        model.compile(optimizer=Adam(learning_rate=learning_rate),
                      loss=weighted_cross_entropy(sample_weights))

        # training
        history = model.fit_generator(generator=train_dataloader,
                                      epochs=epochs)

        # evaluate
        scores = test_set_evaluator(model, path, test_indices, verbose=False)
        scores_cv.append(scores)

        # for progress bar
        pbar.update(1)
        pbar.set_description(f' Fold {i+1} F1: {scores["f1"]}')

# final evaluation
print('Performance model:')
metrics = scores_cv[0].keys()
for metric in metrics:
    if metric == 'confusion': # ignore confusion_matrix
        continue

    mean = np.mean([scores_cv_i[metric] for scores_cv_i in scores_cv])
    std = np.std([scores_cv_i[metric] for scores_cv_i in scores_cv])
    print(f' {metric}: {round(mean, 3)} +- {round(std, 3)} \n')

Starting cross-validation for physical fatigue


  0%|          | 0/5 [00:00<?, ?it/s]

C:\Users\jjung\anaconda3\envs\tf-sklearn-keras\lib\site-packages\keras\engine\training.py:1972: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  warnings.warn('`Model.fit_generator` is deprecated and '


55/55 [==============================] - 47s 364ms/step - loss: 0.4960


 Fold 1 F1: 0.20248495790943366:  20%|██        | 1/5 [01:51<07:27, 111.98s/it]

...

 Fold 4 F1: 0.6027291479628917:  80%|████████  | 4/5 [06:57<01:42, 102.65s/it]

C:\Users\jjung\anaconda3\envs\tf-sklearn-keras\lib\site-packages\keras\engine\training.py:1972: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  warnings.warn('`Model.fit_generator` is deprecated and '


58/58 [==============================] - 52s 378ms/step - loss: 0.3915


 Fold 5 F1: 0.5109519258037749: 100%|██████████| 5/5 [08:52<00:00, 106.58s/it]

Performance model:
 accuracy: 0.473 +- 0.213 

 balanced_accuracy: 0.513 +- 0.024 

 f1: 0.431 +- 0.213 

 recall: 0.473 +- 0.213 

 precision: 0.548 +- 0.255 



In [26]:
# variables
folds = 5
indices = np.arange(N)
epochs = 10
learning_rate = 1e-3
batch_size = 16

# load labels
y = np.empty(N, dtype=int)
for i in range(N):
    y[i] = np.load(path + f'/labels{i}.npy', allow_pickle=True)[variable] # TODO: multiclass

# CV
cv = StratifiedGroupKFold(n_splits=folds)
scores_cv = []

print(f'Starting cross-validation for physical fatigue')
with tqdm(total=folds) as pbar:
    for i, (train_indices, test_indices) in enumerate(cv.split(indices, y, groups=subjects)):
        # training set
        train_dataloader = DataGenerator(path, train_indices, batch_size=batch_size)

        # weights for loss function
        sample_weights = get_weighting_factor(path, train_indices)

        # model
        model = CNN()
        model.compile(optimizer=Adam(learning_rate=learning_rate),
                      loss=weighted_cross_entropy(sample_weights))

        # training
        history = model.fit_generator(generator=train_dataloader,
                                      epochs=epochs)

        # evaluate
        scores = test_set_evaluator(model, path, test_indices, verbose=False)
        scores_cv.append(scores)

        # for progress bar
        pbar.update(1)
        pbar.set_description(f' Fold {i+1} F1: {scores["f1"]}')

# final evaluation
print('Performance model:')
metrics = scores_cv[0].keys()
for metric in metrics:
    if metric == 'confusion': # ignore confusion_matrix
        continue

    mean = np.mean([scores_cv_i[metric] for scores_cv_i in scores_cv])
    std = np.std([scores_cv_i[metric] for scores_cv_i in scores_cv])
    print(f' {metric}: {round(mean, 3)} +- {round(std, 3)} \n')

Starting cross-validation for physical fatigue


  0%|          | 0/5 [00:00<?, ?it/s]

C:\Users\jjung\anaconda3\envs\tf-sklearn-keras\lib\site-packages\keras\engine\training.py:1972: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  warnings.warn('`Model.fit_generator` is deprecated and '


Epoch 1/10
55/55 [==============================] - 43s 343ms/step - loss: 0.6231
Epoch 2/10
55/55 [==============================] - 19s 347ms/step - loss: 0.4274
Epoch 3/10
55/55 [==============================] - 19s 342ms/step - loss: 0.3992
Epoch 4/10
55/55 [==============================] - 19s 342ms/step - loss: 0.4127
Epoch 5/10
55/55 [==============================] - 19s 343ms/step - loss: 0.3623
Epoch 6/10
55/55 [==============================] - 19s 343ms/step - loss: 0.3273
Epoch 7/10
55/55 [==============================] - 19s 343ms/step - loss: 0.2868
Epoch 8/10
55/55 [==============================] - 19s 343ms/step - loss: 0.2965
Epoch 9/10
55/55 [==============================] - 19s 343ms/step - loss: 0.3284
Epoch 10/10
55/55 [==============================] - 19s 343ms/step - loss: 0.2785


 Fold 1 F1: 0.31036017939636895:  20%|██        | 1/5 [04:43<18:54, 283.66s/it]

...

 Fold 4 F1: 0.6525726392251816:  80%|████████  | 4/5 [19:33<04:52, 292.55s/it]

C:\Users\jjung\anaconda3\envs\tf-sklearn-keras\lib\site-packages\keras\engine\training.py:1972: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  warnings.warn('`Model.fit_generator` is deprecated and '


Epoch 1/10
58/58 [==============================] - 45s 325ms/step - loss: 0.3169
Epoch 2/10
58/58 [==============================] - 19s 324ms/step - loss: 0.2663
Epoch 3/10
58/58 [==============================] - 19s 329ms/step - loss: 0.2144
Epoch 4/10
58/58 [==============================] - 19s 324ms/step - loss: 0.2301
Epoch 5/10
58/58 [==============================] - 20s 337ms/step - loss: 0.2304
Epoch 6/10
58/58 [==============================] - 19s 326ms/step - loss: 0.1637
Epoch 7/10
58/58 [==============================] - 19s 327ms/step - loss: 0.1645
Epoch 8/10
58/58 [==============================] - 19s 326ms/step - loss: 0.1405
Epoch 9/10
58/58 [==============================] - 19s 326ms/step - loss: 0.1486
Epoch 10/10
58/58 [==============================] - 19s 327ms/step - loss: 0.1258


 Fold 5 F1: 0.5831730704799175: 100%|██████████| 5/5 [24:02<00:00, 288.48s/it]

Performance model:
 accuracy: 0.506 +- 0.149 

 balanced_accuracy: 0.537 +- 0.057 

 f1: 0.519 +- 0.115 

 recall: 0.506 +- 0.149 

 precision: 0.669 +- 0.144 



In [28]:
# 5fold CV
from sklearn.model_selection import StratifiedKFold, LeaveOneGroupOut, StratifiedGroupKFold
SEED = 42

# variables
folds = 5
indices = np.arange(N)
epochs = 1
learning_rate = 1e-3
batch_size = 16

# load labels
y = np.empty(N, dtype=int)
for i in range(N):
    y[i] = np.load(path + f'/labels{i}.npy', allow_pickle=True)[variable] # TODO: multiclass

# CV
cv = StratifiedKFold(n_splits=folds, shuffle=True, random_state=SEED)
scores_cv = []

print(f'Starting cross-validation for physical fatigue')
with tqdm(total=folds) as pbar:
    for i, (train_indices, test_indices) in enumerate(cv.split(indices, y, groups=subjects)):
        # training set
        train_dataloader = DataGenerator(path, train_indices, batch_size=batch_size)

        # weights for loss function
        sample_weights = get_weighting_factor(path, train_indices)
        print(sample_weights)

        # model
        model = CNN()
        model.compile(optimizer=Adam(learning_rate=learning_rate),
                      loss=weighted_cross_entropy(sample_weights))

        # training
        history = model.fit_generator(generator=train_dataloader,
                                      epochs=epochs)

        # evaluate
        scores = test_set_evaluator(model, path, test_indices, verbose=False)
        scores_cv.append(scores)

        # for progress bar
        pbar.update(1)
        pbar.set_description(f' Fold {i+1} F1: {scores["f1"]}')

# final evaluation
print('Performance model:')
metrics = scores_cv[0].keys()
for metric in metrics:
    if metric == 'confusion': # ignore confusion_matrix
        continue

    mean = np.mean([scores_cv_i[metric] for scores_cv_i in scores_cv])
    std = np.std([scores_cv_i[metric] for scores_cv_i in scores_cv])
    print(f' {metric}: {round(mean, 3)} +- {round(std, 3)} \n')

Starting cross-validation for physical fatigue


  0%|          | 0/5 [00:00<?, ?it/s]

0.32697547683923706


C:\Users\jjung\anaconda3\envs\tf-sklearn-keras\lib\site-packages\keras\engine\training.py:1972: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  warnings.warn('`Model.fit_generator` is deprecated and '


60/60 [==============================] - 48s 341ms/step - loss: 0.4109


...

0.32833787465940056


C:\Users\jjung\anaconda3\envs\tf-sklearn-keras\lib\site-packages\keras\engine\training.py:1972: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  warnings.warn('`Model.fit_generator` is deprecated and '


60/60 [==============================] - 48s 352ms/step - loss: 0.4183


 Fold 5 F1: 0.6566358322321035: 100%|██████████| 5/5 [07:55<00:00, 95.12s/it]

Performance model:
 accuracy: 0.741 +- 0.027 

 balanced_accuracy: 0.528 +- 0.023 

 f1: 0.672 +- 0.019 

 recall: 0.741 +- 0.027 

 precision: 0.7 +- 0.082 



In [30]:
# 5fold CV
from sklearn.model_selection import StratifiedKFold, LeaveOneGroupOut, StratifiedGroupKFold
SEED = 42

# variables
folds = 5
indices = np.arange(N)
epochs = 10
learning_rate = 1e-3
batch_size = 16

# load labels
y = np.empty(N, dtype=int)
for i in range(N):
    y[i] = np.load(path + f'/labels{i}.npy', allow_pickle=True)[variable] # TODO: multiclass

# CV
cv = StratifiedKFold(n_splits=folds, shuffle=True, random_state=SEED)
scores_cv = []

print(f'Starting cross-validation for physical fatigue')
with tqdm(total=folds) as pbar:
    for i, (train_indices, test_indices) in enumerate(cv.split(indices, y, groups=subjects)):
        # training set
        train_dataloader = DataGenerator(path, train_indices, batch_size=batch_size)

        # weights for loss function
        sample_weights = get_weighting_factor(path, train_indices)
        print(sample_weights)

        # model
        model = CNN()
        model.compile(optimizer=Adam(learning_rate=learning_rate),
                      loss=weighted_cross_entropy(sample_weights))

        # training
        history = model.fit_generator(generator=train_dataloader,
                                      epochs=epochs)

        # evaluate
        scores = test_set_evaluator(model, path, test_indices, verbose=False)
        scores_cv.append(scores)

        # for progress bar
        pbar.update(1)
        pbar.set_description(f' Fold {i+1} F1: {scores["f1"]}')

# final evaluation
print('Performance model:')
metrics = scores_cv[0].keys()
for metric in metrics:
    if metric == 'confusion': # ignore confusion_matrix
        continue

    mean = np.mean([scores_cv_i[metric] for scores_cv_i in scores_cv])
    std = np.std([scores_cv_i[metric] for scores_cv_i in scores_cv])
    print(f' {metric}: {round(mean, 3)} +- {round(std, 3)} \n')

Starting cross-validation for physical fatigue


  0%|          | 0/5 [00:00<?, ?it/s]

0.32697547683923706


C:\Users\jjung\anaconda3\envs\tf-sklearn-keras\lib\site-packages\keras\engine\training.py:1972: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  warnings.warn('`Model.fit_generator` is deprecated and '


Epoch 1/10
60/60 [==============================] - 50s 348ms/step - loss: 0.4225
Epoch 2/10
60/60 [==============================] - 21s 347ms/step - loss: 0.3246
Epoch 3/10
60/60 [==============================] - 21s 344ms/step - loss: 0.2976
Epoch 4/10
60/60 [==============================] - 21s 345ms/step - loss: 0.2703
Epoch 5/10
60/60 [==============================] - 21s 347ms/step - loss: 0.2734
Epoch 6/10
60/60 [==============================] - 21s 350ms/step - loss: 0.2184
Epoch 7/10
60/60 [==============================] - 21s 350ms/step - loss: 0.2221
Epoch 8/10
60/60 [==============================] - 21s 347ms/step - loss: 0.2537
Epoch 9/10
60/60 [==============================] - 21s 350ms/step - loss: 0.2046
Epoch 10/10
60/60 [==============================] - 21s 349ms/step - loss: 0.1724


...

0.32833787465940056


C:\Users\jjung\anaconda3\envs\tf-sklearn-keras\lib\site-packages\keras\engine\training.py:1972: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  warnings.warn('`Model.fit_generator` is deprecated and '


Epoch 1/10
60/60 [==============================] - 49s 362ms/step - loss: 0.4804
Epoch 2/10
60/60 [==============================] - 22s 365ms/step - loss: 0.3138
Epoch 3/10
60/60 [==============================] - 22s 370ms/step - loss: 0.2866
Epoch 4/10
60/60 [==============================] - 22s 364ms/step - loss: 0.2728
Epoch 5/10
60/60 [==============================] - 22s 361ms/step - loss: 0.2379
Epoch 6/10
60/60 [==============================] - 22s 358ms/step - loss: 0.2508
Epoch 7/10
60/60 [==============================] - 22s 365ms/step - loss: 0.2354
Epoch 8/10
60/60 [==============================] - 22s 372ms/step - loss: 0.2096
Epoch 9/10
60/60 [==============================] - 22s 372ms/step - loss: 0.2239
Epoch 10/10
60/60 [==============================] - 22s 372ms/step - loss: 0.1991


 Fold 5 F1: 0.7603761841250299: 100%|██████████| 5/5 [24:38<00:00, 295.68s/it]

Performance model:
 accuracy: 0.684 +- 0.201 

 balanced_accuracy: 0.66 +- 0.076 

 f1: 0.657 +- 0.244 

 recall: 0.684 +- 0.201 

 precision: 0.789 +- 0.019 



In [29]:
# LOSO

# variables
folds = len(np.unique(subjects))
indices = np.arange(N)
epochs = 1
learning_rate = 1e-3
batch_size = 16

# load labels
y = np.empty(N, dtype=int)
for i in range(N):
    y[i] = np.load(path + f'/labels{i}.npy', allow_pickle=True)[variable] # TODO: multiclass

# CV
cv = LeaveOneGroupOut()
scores_cv = []

print(f'Starting cross-validation for physical fatigue')
with tqdm(total=folds) as pbar:
    for i, (train_indices, test_indices) in enumerate(cv.split(indices, y, groups=subjects)):
        # training set
        train_dataloader = DataGenerator(path, train_indices, batch_size=batch_size)

        # weights for loss function
        sample_weights = get_weighting_factor(path, train_indices)

        # model
        model = CNN()
        model.compile(optimizer=Adam(learning_rate=learning_rate),
                      loss=weighted_cross_entropy(sample_weights))

        # training
        history = model.fit_generator(generator=train_dataloader,
                                      epochs=epochs)

        # evaluate
        scores = test_set_evaluator(model, path, test_indices, verbose=False)
        scores_cv.append(scores)

        # for progress bar
        pbar.update(1)
        pbar.set_description(f' Fold {i+1} F1: {scores["f1"]}')

# final evaluation
print('Performance model:')
metrics = scores_cv[0].keys()
for metric in metrics:
    if metric == 'confusion': # ignore confusion_matrix
        continue

    mean = np.mean([scores_cv_i[metric] for scores_cv_i in scores_cv])
    std = np.std([scores_cv_i[metric] for scores_cv_i in scores_cv])
    print(f' {metric}: {round(mean, 3)} +- {round(std, 3)} \n')

Starting cross-validation for physical fatigue


  0%|          | 0/24 [00:00<?, ?it/s]

C:\Users\jjung\anaconda3\envs\tf-sklearn-keras\lib\site-packages\keras\engine\training.py:1972: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  warnings.warn('`Model.fit_generator` is deprecated and '


74/74 [==============================] - 52s 340ms/step - loss: 0.3964


 Fold 1 F1: 0.38252873563218387:   4%|▍         | 1/24 [01:15<28:58, 75.57s/it]

...

 Fold 23 F1: 0.7670712857134943:  96%|█████████▌| 23/24 [29:10<01:26, 86.66s/it] 

C:\Users\jjung\anaconda3\envs\tf-sklearn-keras\lib\site-packages\keras\engine\training.py:1972: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  warnings.warn('`Model.fit_generator` is deprecated and '


71/71 [==============================] - 53s 344ms/step - loss: 0.4638


 Fold 24 F1: 0.35940224159402245: 100%|██████████| 24/24 [30:29<00:00, 76.23s/it]

Performance model:
 accuracy: 0.52 +- 0.266 

 balanced_accuracy: 0.539 +- 0.215 

 f1: 0.483 +- 0.299 

 recall: 0.52 +- 0.266 

 precision: 0.63 +- 0.334 

